In [1]:
import pandas as pd
import requests
from collections import defaultdict
from pathlib import Path

In [2]:
year = 2022
leaderboard_id = 976545
token_path = Path.home() / '.config/aocd/token'

In [3]:
with open(token_path) as f:
    token = f.read().strip()

cookies = {'session': token}
headers = {'User-Agent': 'github.com/attoPascal'}
response = requests.get(f'https://adventofcode.com/{year}/leaderboard/private/view/{leaderboard_id}.json', cookies=cookies, headers=headers).json()

In [4]:
time_diffs = defaultdict(lambda: defaultdict(str))

for id, member in response['members'].items():
    name = member['name'] if member['name'] else id
    for day, times in member['completion_day_level'].items():
        if '1' in times and '2' in times:
            time_diffs[name][int(day)] = times['2']['get_star_ts'] - times['1']['get_star_ts']

In [5]:
df = pd.DataFrame.from_dict(time_diffs, orient='index', dtype='Int64')
df.sort_index(key=lambda x: x.str.lower(), inplace=True)
df.sort_index(axis=1, inplace=True)
df

,1,2,3,4,5,6
2316524,3353,<NA>,<NA>,<NA>,<NA>,<NA>
740740,1001,976,7452,325,47268,148
999754,434,1572,680,750,770,<NA>
Alex Ji,914,530,<NA>,<NA>,<NA>,<NA>
attoPascal,117,513,444,267,394,40
Azratosh,120,103,247,181,456,49
benjo121ben,419,952,222,1083,325,107
Benni S,475,788,2688,276,628,<NA>
christoph-det,327,1989,4482,166,988,140
cluosh,118,9870,251,72,84,38


In [6]:
num_members, num_days = df.shape
points = pd.Series(0, index=df.index, dtype=int)

for day in range(num_days):
    points_awarded = num_members
    for name, time in df[day+1].sort_values().items():
        if not pd.isna(time):
            points[name] += points_awarded
            points_awarded -= 1

In [7]:
# part 2 leaderboard
leaderboard = points.sort_values(ascending=False).to_frame().reset_index()
leaderboard.index += 1
leaderboard.columns = ['name', 'pts']
leaderboard = leaderboard[['pts', 'name']]
leaderboard.head(20)


,pts,name
1,291,Azratosh
2,278,TechnikTobi
3,276,RaphaelTarita
4,273,msonnberger
5,273,cluosh
6,273,David Mikan
7,268,attoPascal
8,261,felpower
9,261,Hainkinho
10,261,S-ecki


In [8]:
# seconds for each part 2
df.loc['attoPascal'].to_frame()

,attoPascal
1,117
2,513
3,444
4,267
5,394
6,40


In [9]:
# seconds for latest part 2
df.iloc[:, -1].dropna().sort_values().to_frame()

,6
S-ecki,36
cluosh,38
David Mikan,38
attoPascal,40
TechnikTobi,44
JohnBryte,44
tomato-alex,45
Hainkinho,48
Azratosh,49
RaphaelTarita,53


In [10]:
# sum for all parts 2
df.sum(axis=1, skipna=False).dropna().sort_values().to_frame() # / 3600

,0
Azratosh,1156
msonnberger,1431
TechnikTobi,1548
RaphaelTarita,1616
NotsoblackReaper,1768
attoPascal,1775
David Mikan,1782
thoosequa,1846
S-ecki,1945
Hainkinho,2013
